In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas
import os.path
import rpy2
import scorecardpy as sc
from scorecardpy import info_value 

In [2]:
f = os.path.expanduser( "~/Documents/Vodacom/data/df_raw_final.pkl" )

df = pd.read_pickle(f)

In [3]:
#new dataframe after dropping all values with NaN
dfclean = df.dropna( subset = ['target_label'] )

In [4]:
#replace all strings "yes", "no", "maybe" with integers 0, 1, 2 
dfclean = dfclean.replace( to_replace = "yes", value = 1 )
dfclean = dfclean.replace( to_replace = "no", value = 0 )
dfclean = dfclean.replace( to_replace = "maybe", value = 2 )

In [5]:
#remove all rows with target_label = 2 ("maybe")
dfclean = dfclean[ dfclean.target_label != 2 ]

In [6]:
#remove column geometry due to string error with conversion
dfclean_nogeo = dfclean.drop( columns = "geometry" )
dfclean_nogeo['randfeature'] = np.random.normal(size = dfclean_nogeo.shape[0])

plt.hist(dfclean_nogeo.randfeature, 100)
plt.show()

Column 'geometry' in dataframe causes unwanted errors.
Initialize new dataframe without the 'geometry' column.
Assumption that the 'geometry' column causes more issues than has relevance for model.

In [7]:
Feat_typ = {}
Num_typ = {}

num_list = []
cat_list = []

j = 1

# for loop to identify feature types

for i, feature in enumerate( dfclean_nogeo ):
    
    if ( dfclean_nogeo[ feature ].dtype == 'float64' or dfclean_nogeo[ feature ].dtype == 'int64'):
        
        value_list = dfclean_nogeo[feature].values
        length_values_list = []
        
# for loop identifying if all items in a given feature have the same number of characters
# if they do, identify feature as a category
        
        for item in value_list:
            
            length_values_list.append( len( str( item ) ) )

# CODE = true if number of characters in values of all features are the same            
            
        CODE = ( len( set( length_values_list ) ) == 1)
        
    if (dfclean_nogeo[feature].dtype == 'object' or CODE):     

        Feat_typ[feature] = [i,"cat"]
        cat_list.append(feature)
        
    else:
        
        Feat_typ[feature] = [i, "num"]
        Num_typ[feature] = [j, "num"]
        num_list.append(feature)
        j = j + 1
     
    # adding the 'target_label' column to num_list    
    
    if feature == "target_label":

        num_list.append( feature )


big explanation here probs

In [8]:
%load_ext rpy2.ipython

Activates the use of R in the Jupyter Notebook

In [9]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'

Brute force fix for kernel dying on Mac, Error15

In [10]:
%%R -i dfclean_nogeo -i num_list -o cuts

library(smbinning)
library(foreach)
library(doParallel)


comb <- function(...){
    mapply('cbind', ..., SIMPLIFY = FALSE)
}

no_cores <- detectCores()
registerDoParallel(cores = 4)

numg_list = c()
cutsgood = c()

Features <- names(dfclean_nogeo)[1:ncol(dfclean_nogeo)]

i = 1
j = 1

ptime <- system.time({

 cuts = foreach( col = Features, .combine = comb, .packages = 'smbinning')  %dopar%  {

    sbin = smbinning( dfclean_nogeo, 'target_label', col )
            
       if ( !is.na( sbin[ 'bands' ] ) ) {
        
            cutsgood[[i]] <- sbin$bands[ -length( sbin$bands ) ]
            numg_list[i] = col
            i = i + 1
           
        } 
     
        # adding target_label to numg_list
        if ( col == "target_label" ) {

            numg_list[i] = col

        }
     
     return(list(numg_list, cutsgood))
     
    }
    
})[3]
print(paste("Time taken for smbinning =", ptime, "seconds"))

R[write to console]: Loading required package: sqldf

R[write to console]: Loading required package: gsubfn

R[write to console]: Loading required package: proto

R[write to console]: Loading required package: RSQLite

R[write to console]: Loading required package: partykit

R[write to console]: Loading required package: grid

R[write to console]: Loading required package: libcoin

R[write to console]: Loading required package: mvtnorm

R[write to console]: Loading required package: Formula

R[write to console]: Loading required package: iterators

R[write to console]: Loading required package: parallel



[1] "Time taken for smbinning = 13.413 seconds"


In [11]:
# create a list for numg_list from an R vector
cuts = list(cuts)


In [22]:
np.save('cuts', cuts)

In [12]:
# creating dictionary of value cutsgood
cutsg_dic = dict( zip( cuts[0], cuts[1] ) )

{'EA_CODE': R object with classes: ('numeric',) mapped to:
 [16310023.000000, 29910719.000000, 79710794.000000, 79713705.000000, 79912661.000000, 87410237.000000, 96811076.000000],
 'SAL_CODE': R object with classes: ('numeric',) mapped to:
 [1630082.000000, 2991701.000000, 7970216.000000, 7972659.000000, 7991309.000000, 8740705.000000, 9681005.000000],
 'W_M_INC17': R object with classes: ('numeric',) mapped to:
 [0.000000, 0.000000, 203041.157100, 300232.962400],
 'P_NLI1': R object with classes: ('numeric',) mapped to:
 [0.000000, 0.005400, 0.095300, 0.144900, 0.200200, 0.459400],
 'P_NLI2': R object with classes: ('numeric',) mapped to:
 [0.000000, 0.001100, 0.031800, 0.080300, 0.113500],
 'P_NLI3': R object with classes: ('numeric',) mapped to:
 [0.000000, 0.003300, 0.143800],
 'P_NLI4': R object with classes: ('numeric',) mapped to:
 [0.000000, 0.000000, 0.016900, 0.034900, 0.055200, 0.155400],
 'P_NLI5': R object with classes: ('numeric',) mapped to:
 [0.000000, 0.000000, 0.0105

The woebin function creates IV tables for all numerical features in dataframe. It uses smbinning's cut values for the bins for all numerical features within cutsg_dic, all other numerical features have bins cut by the woebin function. 
This is due to the assumption that smbinning utilizes a more accurate cut value for the bins and woebinning is the second best option available to us. 

In [13]:
num_iv = sc.woebin( dfclean_nogeo.loc[ :, num_list ], y = "target_label", breaks_list = cutsg_dic, positive = 0)

[INFO] creating woe binning ...


/opt/anaconda3/lib/python3.7/site-packages/scorecardpy/condition_fun.py:113: UserWarning: The positive value in "target_label" was replaced by 1 and negative value by 0.
  warnings.warn("The positive value in \"{}\" was replaced by 1 and negative value by 0.".format(y))
/opt/anaconda3/lib/python3.7/site-packages/scorecardpy/woebin.py:725: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  binning = pd.concat(bin_list, keys=bin_list.keys()).reset_index()\
/opt/anaconda3/lib/python3.7/site-packages/scorecardpy/woebin.py:725: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [14]:
# %%R -i dfclean_nogeo -i num_list -o cutsgood -o cutsbad -o numg_list -o numb_list

# library(smbinning)

# cutsgood = list()
# cutsbad = list()
# numg_list = list()
# numb_list = list()

# i = 1
# j = 1

# ptime <- system.time({

#     for ( col in num_list ) {

#         sbin = smbinning( dfclean_nogeo, 'target_label', col )

#         if ( !is.na( sbin[ 'bands' ] ) ) {

#             cutsgood[[i]] <- sbin$bands[ -length( sbin$bands ) ]
#             numg_list[i] = col
#             i = i + 1

#         } else {

#             cutsbad[[j]] = NA
#             numb_list[i] = col
#             j = j + 1

#         }

#         # adding target_label to numb_list and numg_list
#         if ( col == "target_label" ) {

#             numb_list[i] = col
#             numg_list[i] = col

#         }


#     }
    
# })[3]

# print(paste("Time taken for smbinning =", ptime, "seconds"))

In [15]:
# # keep cutsgood as a listvector to add to dictionary
# # create a list for numg_list from an R vector
# numg_list = list( zip( *numg_list ) )
# numg_list = list( numg_list[0] )

# # creating dictionary of values numg_list and cutsgood
# cutsg_dic = dict( zip( numg_list, cutsgood ) )

# num_iv = sc.woebin( dfclean_nogeo.loc[ :, num_list ], y = "target_label", breaks_list = cutsg_dic, positive = 0)


In [16]:
Feat_sum = pd.DataFrame( dfclean_nogeo.dtypes )

i = 0

for feature in dfclean_nogeo:
    
    Feat_sum.loc[ feature,"Unique" ] = dfclean_nogeo[ feature ].nunique()
    Feat_sum.loc[ feature, "NaNs" ] = dfclean_nogeo[ feature ].isna().sum()
    i = i + 1

Creating a dataframe to summarize each feature

In [17]:
# IV values of all categorical features in dataframe
cat_iv = sc.info_value.iv( dfclean_nogeo.loc[ :, cat_list ], y = "target_label" )

In [18]:
# changing the headings of the dataframe cat_iv
cat_iv = cat_iv.rename( columns = { 'variable':'', 'info_value': 'Total_IV' } )

cat_iv = cat_iv.set_index('')

In [19]:
# adding iv values of numerical features to summary dataframe
for feature in dfclean_nogeo.loc[ :, num_list ]:
    
    if ( feature != 'target_label') :
        
        Feat_sum.loc[ feature,'Total_IV' ] = num_iv[ feature ].total_iv[ 1 ]

In [20]:
i = 1

# adding iv values of categorical features to summary datframe
for feature in dfclean_nogeo.loc[ :, cat_list ]:
    
    if ( feature != 'target_label' ) :
        
        Feat_sum.loc[ feature,'Total_IV' ] = cat_iv.loc[ feature, 'Total_IV' ]
        
Feat_sum

,0,Unique,NaNs,Total_IV
EA_CODE,float64,2129.0,0.0,0.010996
SAL_CODE,float64,1797.0,0.0,0.215172
SP_NAME,object,653.0,0.0,2.377328
SP_NBH_ID,object,669.0,0.0,2.327769
MP_NAME,object,172.0,0.0,2.298901
...,...,...,...,...
D_MIGR_OUT_KM2,float64,2006.0,0.0,1.538771
D_MIGR_IN_KM2,float64,2059.0,0.0,1.163693
sales_label,float64,2.0,1121.0,5.924251
target_label,int64,2.0,0.0,NaN


In [21]:
Feat_sum.sort_values('Total_IV', ascending = False).head(40)

,0,Unique,NaNs,Total_IV
human_label,float64,3.0,992.0,6.237010
sales_label,float64,2.0,1121.0,5.924251
sales_psk,float64,1138.0,991.0,4.991273
avg_monthly_sales_psk,float64,1138.0,991.0,3.375237
P_NLI1,float64,1696.0,0.0,2.447159
SP_NAME,object,653.0,0.0,2.377328
SP_NBH_ID,object,669.0,0.0,2.327769
computer_perc,float64,1516.0,18.0,2.312888
MP_NAME,object,172.0,0.0,2.298901
AREA_KM2,float64,2129.0,0.0,2.115097
